In [1]:
TENSORFLOW = 1
GLUON = 1

import os  
if GLUON:
    os.environ['CUDA_VISIBLE_DEVICES']='0'

if TENSORFLOW:
    import tensorflow as tf

    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            tf.config.set_visible_devices(gpus[0], 'GPU')
            tf.config.set_logical_device_configuration(
                gpus[0],
                [tf.config.LogicalDeviceConfiguration(memory_limit=46000)])
            logical_gpus = tf.config.list_logical_devices('GPU')
            print(gpus, "Physical GPUs,", logical_gpus, "Logical GPUs")
        except RuntimeError as e:
            print(e)


2024-04-11 12:45:22.434803: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-11 12:45:22.479640: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-11 12:45:23.271072: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')] Physical GPUs, [LogicalDevice(name='/device:GPU:0', device_type='GPU')] Logical GPUs


2024-04-11 12:45:25.101015: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46000 MB memory:  -> device: 0, name: Quadro RTX 8000, pci bus id: 0000:3b:00.0, compute capability: 7.5


In [2]:
import warnings, pandas as pd, requests, mlflow, sys, os, logging, numpy as np
from automlwrapper import AutoMLWrapper, SedarDataLoader
from sklearn.model_selection import train_test_split

from sedarapi import SedarAPI

logging.getLogger('automlwrapper').setLevel(logging.ERROR)

np.random.seed(1313)

TIME_LIMIT = 60 * 15
AUTOKERAS_TRAILS = 20
AUTOKERAS_EPOCHS = 30

Using TensorFlow backend
WARNING AutoSklearn could not be mported. It might not b available in this environment. Err: 
 No module named 'autosklearn'.
WARNING AutoSklearn could not be imported. It might not be available in this environment. Err: 
 No module named 'autosklearn'.


In [3]:
USE_MLFLOW = False

MLFLOW_URI = 'http://192.168.220.107:6798'
MLFLOW_EXPID = '1'
if USE_MLFLOW:
    mlflow.set_tracking_uri(MLFLOW_URI) 
    mlflow.set_experiment(experiment_id=MLFLOW_EXPID)

In [4]:
SEDAR_URI = 'http://192.168.220.107:5000'

SEDAR = SedarAPI(SEDAR_URI)
DataLoader = SedarDataLoader(SEDAR)

In [5]:
zip_location = './tmp/data/opticalv1'

# download data to the above location

In [6]:
!curl --create-dirs -O --output-dir \
./tmp/data/opticalv1 \
https://gitlab.com/mibbels/automlwrapperdata/-/raw/main/image-classification/surface_defects_image_classification.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  823M  100  823M    0     0  89.4M      0  0:00:09  0:00:09 --:--:-- 93.2M


## Read the image classification dataset from files
#### Same code utilizing the SEDAR API:
```python
DataLoader.query_data('<workspace id>',
                      '<dataset id>',
                      file_save_location = zip_location)
```

In [7]:
optical, class_map = DataLoader.classificationAsDataFrame(zip_location, zip_location + '/unzip')

In [8]:
def create_few_shot(df, num_shots = 10, target_column = 'label'):

    few_shot_df = pd.DataFrame()
    
    for label in df[target_column].unique():
        sampled_df = df[df[target_column] == label].sample(
            n=min(
                num_shots,
                len(df[df[target_column] == label])
            ),
            replace=False
        )
        few_shot_df = pd.concat([few_shot_df, sampled_df])

    return few_shot_df


# create artificial few-shot datasets @ 5 and @ 20 shots

In [9]:
X = optical.drop(columns=['label'])  
y = optical['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

train_few5 = create_few_shot(train, 5, 'label')
#test_few5 = optical[~optical.index.isin(train_few5.index)]

train_few20 = create_few_shot(train, 20, 'label')
#test_few20 = optical[~optical.index.isin(train_few20.index)]


In [10]:
train_few5.count()

image    20
label    20
dtype: int64

In [11]:
train_few20.count()

image    80
label    80
dtype: int64

In [12]:
test.count()

image    79
label    79
dtype: int64

# Optimizing a single predictor using AutoGluon

In [ ]:
def wrapper_gluon(train_data, eval_metric):
    
    wrapper = AutoMLWrapper('autogluon')
    wrapper.AllowExtraHyperparameters({'fit': ['hyperparameters'],
                                       'constructor': ['num_classes']})
    wrapper.Train(
        train_data=train_data,
        target_column='label',
        task_type='classification',
        data_type='image',
        problem_type='multiclass',
        hyperparameters={
            'time_limit': TIME_LIMIT,
            'preset' : 'medium_quality',
            'eval_metric' : eval_metric,
            'validation_metric':eval_metric,
            'verbosity':1,
                            }    # comment out if extra parameters below
            #####
            #    Extra parameter were specified. The wrapper will now accept them.
            # Parameters not present in 'AllowExtraHyperparameters' or the .yaml 
            # file are still ignored.
            #####
            #'hyperparameters':{"optimization.max_epochs": epochs},
            #'num_classes':4
            #          }     # uncomment if extra parameters 
            
    )
    
    return wrapper

### @5

In [ ]:
%%time

w_med5 = wrapper_gluon(train_few5, 'f1_macro')


In [ ]:
%%time

re5 = w_med5.Evaluate(test, target_column=None)#, **kwargs)
print(re5)

### @20

In [ ]:
%%time
w_med20 = wrapper_gluon(train_few20, 'f1_macro')


In [ ]:
%%time

re20 = w_med20.Evaluate(test, target_column=None)#, **kwargs)
print(re20)

# Neural network optimization using the Wrapper with AutoKeras

- you need to specify the batch size manually for 5-shot , default exceeds 20 samples 

In [ ]:
def wrapper_keras(train_data, epochs, trials, eval_metric, batch):
    import keras_tuner 

    wrapper = AutoMLWrapper('autokeras')
    wrapper.AllowExtraHyperparameters({'fit':['batch_size']})
    wrapper.Train(
        train_data=train_data,
        target_column='label',
        task_type='classification',
        data_type='image',
        problem_type='multiclass',
        hyperparameters={
            'evaluation_metric':[eval_metric, 'acc', 'AUC', 'Precision', 'Recall'],
            'epochs':epochs,
            'num_trials':trials,
            'objective_metric':keras_tuner.Objective("val_f1_score", direction="max"),
            'batch_size':batch
        }
    )
    if USE_MLFLOW:
         additional_tags = {
         "workspace":"DefaultWorkspace,13b4787c3e454649aa05a4cd680edc37",
         "notebook": "glass classifiaction PB,854c94fd-1898-4f03-b0c6-b60b5571b102",
         "is_public":"true",
         "datasets":"glass_schema|986f2e837ca44f3e8c0ee7d2dc0c4287",
         "username":"mibbels",
         }
         n = 2
         wrapper.MlflowUploadTopN(n, additional_tags)
        
    return wrapper

### @5

In [ ]:
%%time

f1 = tf.keras.metrics.F1Score(
        average='macro', threshold=None, name='f1_score', dtype=None
    )

n_epoch = AUTOKERAS_EPOCHS
n_trial = AUTOKERAS_TRAILS
w_keras = wrapper_keras(train_few5, n_epoch, n_trial, f1, batch=8)

In [ ]:
re = w_keras.Evaluate(test, target_column='label', return_dict = True)
print(re)

### @20

In [ ]:
%%time

f1 = tf.keras.metrics.F1Score(
        average='macro', threshold=None, name='f1_score', dtype=None
    )

n_epoch = AUTOKERAS_EPOCHS
n_trial = AUTOKERAS_TRAILS
w_keras = wrapper_keras(train_few20, n_epoch, n_trial, f1, batch=16)

In [ ]:
re = w_keras.Evaluate(test, target_column='label', return_dict = True)
print(re)

# Optimizing a few-shot predictor using AutoGluon

In [ ]:
def wrapper_few(train_data, eval_metric):

    wrapper = AutoMLWrapper('autogluon')
    wrapper.Train(
        train_data=train_data,
        target_column='label',
        task_type='few-shot-classification',
        data_type='image',
        problem_type='multiclass',
        hyperparameters={
            'time_limit': TIME_LIMIT,
            'preset' : 'best_quality',
            'eval_metric' : eval_metric,
            'validation_metric':eval_metric,
            'verbosity':1,
            }
    )
    
    return wrapper

### @5

In [ ]:
%%time

w_few5 = wrapper_few(train_few5, 'f1_macro')


In [ ]:
re5 = w_few5.Evaluate(test, target_column=None)
print(re5)

### @20

In [ ]:
%%time

w_few20 = wrapper_few(train_few20, 'f1_macro')


In [ ]:
re5 = w_few20.Evaluate(test, target_column=None)
print(re5)

---------
# Evaluating the LLMs capabilities when creating image classification code

# **Trial 1**: A custom prompt only tackling  data loading 
I have an image classification dataset contained in a pandas dataframe.
The dataframe is called opticalv1 with columns 'image' and 'label'. 'image' contains the path to the image file and 'label' contains the class label.
The dataset contains 394 images. 

All images are 768px in height, 1024px in width, in RGB mode, and saved as png.
Here is the class distribution for the dataset:
label
0    188
3     56
2     37
1     34

Generate me python code to load this dataset into a format that can be directly used by deep learning libraries like pytorch or tensorflow.

## Hermes was evaluated to provide the code to load the file-paths in the DataFrame into tensors
```python
import pandas as pd
from PIL import Image
import torch
# Load the dataframe:
df = train_few20
# Define a function to preprocess each image:
def preprocess(image):
    # load the image from file
    img = Image.open(image).convert('RGB')
    
    # resize the image to 256x256 pixels (if necessary)
    if img.width != 768 or img.height != 1024:
        img = img.resize((256, 256))#, Image.ANTIALIAS)
    
    # normalize the pixel values to be between -1 and 1 (if necessary)
    if img.mode == 'L':
        img = img.convert('RGB')
    elif img.mode != 'RGB':
        img = img.convert('RGB', colorspace='lab2rgb_reflect')

    # normalize the normalize the pixel values to be between -1 and 1 (if necessary)

    mean = torch.tensor([0.485, 0.456, 0.406])
    std = torch.tensor([0.229, 0.224, 0.225])

    img_tensor = Image.fromarray(img.resize((224, 224)))#, Image.ANTIALIAS))
    normalized_img = (img_tensor / 255.0) * std + mean

    # return the preprocessed image as a PyTorch tensor
    return torch.from_numpy(normalized_img.as_inplace().astype('float32'))

#Applying this function to each row of the dataframe will create a new column called 'image_tensor' that contains the preprocessed images in PyTorch format:

df['image_tensor'] = df['image'].apply(preprocess)
# Convert the dataset into a torch.utils.data.Dataset object to use
```

#### there are some small confusions in the code an the operations and conversions between data types (torch/np/PIL) are not entirely correct 

# **Trial 2**: A custom prompt describing the situation where data has been converted into a numpy array
I have an image classification dataset contained in two numpy arrays X and y. X is of shape (394, 768, 1024, 3) and y is of shape (394,) The dataset thus contains 394 images. All images are 768px in height, 1024px in width, in RGB mode, and saved as png. Here is the class distribution for y:

0 188 3 56 2 37 1 34

Generate me python code to create and train a neural network for this data. Annotate appropriately for understanding and further use. 

## Hermes was able to provide a fitting answer in two parts 
First answer:


```python
    import numpy as np
    from tensorflow import keras
    # Load the data into NumPy arrays.
    X = np.array(x)  # shape (394, 768, 1024, 3)
    y = np.array(y)  # shape (394,)
    #Class labels are one-hot encoded using scikit-learn's LabelEncoder class.
    from sklearn.preprocessing import LabelEncoder
    from sklearn.preprocessing import OneHotEncoder
    
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    y_encoded = y_encoded.reshape(-1, 1)

    onehot_encoder = OneHotEncoder(sparse=False)
    y_encoded = onehot_encoder.fit_transform(y_encoded)  # Then, one-hot encode



    #Create a Keras Sequential model with two convolutional layers and a softmax output layer.
    model = keras.Sequential([
        keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(768,1024, 3)),
        keras.layers.MaxPooling2D((2, 2)),
        
```

Prompt 2: 

Continue building the network and evaluate it on the test data.

Answer 2:
```python
        # Add a second convolutional layer with 64 filters, followed by another max pooling layer.
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(.5),
        keras.layers.Dense(3, activation='softmax')
    ])
        
    #Compile the model with binary cross-entropy loss and Adam optimizer.
    model.compile(optimizer=keras.optimizers.Adam(),
                  loss=keras.losses.BinaryCrossentropy(),
                  loss=keras.losses.CategoricalCrossentropy(), 
                  metrics=[keras.metrics.Accuracy(), f1])
    #Fit the model on the data using epochs parameter to specify number of iterations over the dataset.
    model.fit(X, y_encoded, epochs=50)

    le = LabelEncoder()
    y_test = le.fit_transform(y_test)
    y_test = y_test.reshape(-1, 1)

    onehot_encoder = OneHotEncoder(sparse=False)
    y_test = onehot_encoder.fit_transform(y_test)
    
    print(model.evaluate(X_test, y_test))
```

### Apart from the use of a 3-neuron output and binary cross-entropy, this network is appropriate 

## Applying the LLMs answer

In [13]:
X_train_few5, y_train_few5 = DataLoader.imageDataFrameToNumpyXy(train_few5)
X_train_few20, y_train_few20 = DataLoader.imageDataFrameToNumpyXy(train_few20)

X_test, y_test = DataLoader.imageDataFrameToNumpyXy(test)



In [14]:
f1 = tf.keras.metrics.F1Score(
        average='macro', threshold=None, name='f1_score', dtype=None
    )

def hermes_cnn_numpy(X, y, X_test, y_test):
    import numpy as np
    from tensorflow import keras
    # Load the data into NumPy arrays.
    #X = np.array(x)  # shape (394, 768, 1024, 3)
    #y = np.array(y)  # shape (394,)
    #Class labels are one-hot encoded using scikit-learn's LabelEncoder class.
    from sklearn.preprocessing import LabelEncoder
    from sklearn.preprocessing import OneHotEncoder
    
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    y_encoded = y_encoded.reshape(-1, 1)

    onehot_encoder = OneHotEncoder(sparse=False)
    y_encoded = onehot_encoder.fit_transform(y_encoded)  # Then, one-hot encode



    #Create a Keras Sequential model with two convolutional layers and a softmax output layer.
    model = keras.Sequential([
        keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(768,1024, 3)),
        keras.layers.MaxPooling2D((2, 2)),
        #################################################### Answers split here
        # Add a second convolutional layer with 64 filters, followed by another max pooling layer.
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(.5),
        keras.layers.Dense(4, activation='softmax')     # EDITED   was 3 neurons
    ])
        
    #Compile the model with binary cross-entropy loss and Adam optimizer.
    model.compile(optimizer=keras.optimizers.Adam(),
                  #loss=keras.losses.BinaryCrossentropy(),  # EDITED was BinaryCrossentropy
                  loss=keras.losses.CategoricalCrossentropy(), 
                  metrics=[keras.metrics.Accuracy(), f1])
    #Fit the model on the data using epochs parameter to specify number of iterations over the dataset.
    model.fit(X, y_encoded, epochs=25)

    le = LabelEncoder()
    y_test = le.fit_transform(y_test)
    y_test = y_test.reshape(-1, 1)

    onehot_encoder = OneHotEncoder(sparse=False)
    y_test = onehot_encoder.fit_transform(y_test)
    
    print(model.evaluate(X_test, y_test))

In [18]:
hermes_cnn_numpy(X_train_few5, y_train_few5, X_test, y_test)

Epoch 1/25


/usr/local/automl/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


1/1 [==============================] - 3s 3s/step - loss: 32.6969 - accuracy: 0.1000 - f1_score: 0.4067
Epoch 2/25
1/1 [==============================] - 0s 161ms/step - loss: 42442.2617 - accuracy: 0.6250 - f1_score: 0.1000
Epoch 3/25
1/1 [==============================] - 0s 161ms/step - loss: 40871.6914 - accuracy: 0.6000 - f1_score: 0.0870
Epoch 4/25
1/1 [==============================] - 0s 159ms/step - loss: 11907.4951 - accuracy: 0.6750 - f1_score: 0.2902
Epoch 5/25
1/1 [==============================] - 0s 161ms/step - loss: 7222.6016 - accuracy: 0.7250 - f1_score: 0.4561
Epoch 6/25
1/1 [==============================] - 0s 159ms/step - loss: 9053.6348 - accuracy: 0.7000 - f1_score: 0.3746
Epoch 7/25
1/1 [==============================] - 0s 159ms/step - loss: 5527.4897 - accuracy: 0.7500 - f1_score: 0.4280
Epoch 8/25
1/1 [==============================] - 0s 159ms/step - loss: 1637.0824 - accuracy: 0.7500 - f1_score: 0.5016
Epoch 9/25
1/1 [==============================] - 0s 

/usr/local/automl/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


3/3 [==============================] - 0s 59ms/step - loss: 82.2538 - accuracy: 0.7563 - f1_score: 0.4254
[82.2537612915039, 0.7563291192054749, 0.4253558814525604]


In [20]:
hermes_cnn_numpy(X_train_few20, y_train_few20, X_test, y_test)

/usr/local/automl/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/25
3/3 [==============================] - 2s 193ms/step - loss: 32456.3008 - accuracy: 0.4313 - f1_score: 0.1681
Epoch 2/25
3/3 [==============================] - 1s 193ms/step - loss: 22263.7129 - accuracy: 0.6812 - f1_score: 0.3132
Epoch 3/25
3/3 [==============================] - 1s 213ms/step - loss: 5062.3013 - accuracy: 0.6844 - f1_score: 0.3382
Epoch 4/25
3/3 [==============================] - 1s 204ms/step - loss: 1135.6100 - accuracy: 0.7531 - f1_score: 0.4948
Epoch 5/25
3/3 [==============================] - 1s 211ms/step - loss: 440.9905 - accuracy: 0.7156 - f1_score: 0.4606
Epoch 6/25
3/3 [==============================] - 1s 209ms/step - loss: 52.7818 - accuracy: 0.7094 - f1_score: 0.6434
Epoch 7/25
3/3 [==============================] - 1s 210ms/step - loss: 14.7284 - accuracy: 0.5562 - f1_score: 0.6704
Epoch 8/25
3/3 [==============================] - 1s 211ms/step - loss: 12.1084 - accuracy: 0.4750 - f1_score: 0.6200
Epoch 9/25
3/3 [=============================

/usr/local/automl/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


3/3 [==============================] - 0s 58ms/step - loss: 0.9672 - accuracy: 0.3133 - f1_score: 0.5549
[0.9672031402587891, 0.31329113245010376, 0.554914653301239]
